#Group A1 Thesis: Milkfish Fingerlings Counter Using the Instance Segmentation of YOLOv7 and Custom Dataset.

Version 7 Notebook: This version will be trained using the version 2 of the milkfish fingerlings v2 custom dataset for higher accuracy.

#1. Mounting Google Colab to Google Drive

In [ ]:
#Connecting to Google Drive to save all the progress.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Go to the directory you want to save the changes.
%cd /content/drive/MyDrive/final-thesis-milkfish-fingerlings-counter

In [ ]:
#Creating or checking YOLOv7 directiroy.
#check lang kung merong existing na folder then pag wala di gagawa
import os

if not os.path.isdir("yolov7-instance-segmentation-a1"):
    os.makedirs("yolov7-instance-segmentation-a1")

In [ ]:
#punta sa ginawang folder
%cd yolov7-instance-segmentation-a1

In [ ]:
# #Making sure that GPU is being utilized in runtime.
!nvidia-smi

#2. Installation of YOLOv7

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
# clone YOLOv7 repository
%cd {HOME}
!git clone https://github.com/WongKinYiu/yolov7

# navigate to yolov7 directory and checkout u7 branch of YOLOv7 - this is hash of lates commit from u7 branch as of 12/21/2022
%cd {HOME}/yolov7
!git checkout 44f30af0daccb1a3baecc5d80eae22948516c579

In [ ]:
#Install Requirements
%cd {HOME}/yolov7/seg
!pip install --upgrade pip
!pip install -r requirements.txt

In [ ]:
# download COCO starting checkpoint to yolov7/seg directory
%cd {HOME}/yolov7/seg
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-seg.pt

WEIGHTS_PATH = f"{HOME}/yolov7/seg/yolov7-seg.pt"

In [ ]:
# WEIGHTS_PATH = f"{HOME}/yolov7/seg/yolov7-seg.pt"

#3. Connecting to Roboflow for extracting the custom dataset (milkfish fingerlings).

In [ ]:
#Roboflow API Key for v2thesisdataset = "vIzSeL9qfZA33Fw4EzNE"
from getpass import getpass

# copy your API KEY from 
api_key = getpass('Enter YOUR_API_KEY secret value: ')

In [ ]:
%cd {HOME}/yolov7/seg

!pip install roboflow --quiet

from roboflow import Roboflow

rf = Roboflow(api_key=api_key)
project = rf.workspace("v2thesisdataset").project("v7-milkfish-fingerlings-dataset-with-augmentation")
dataset = project.version(1).download("yolov7")

In [ ]:
#check dataset directory
print(dataset.location)

#4. Start the custom training for milkfish fingerlings counter using the Instance Segmentation of YOLOv7, specifically the u7 branch not the mask branch.

In [ ]:
%cd {HOME}/yolov7/seg
!python segment/train.py --batch 11 \
 --epochs 120 \
 --data {dataset.location}/data.yaml \
 --weights $WEIGHTS_PATH \
 --device 0 \
 --name 120epochs-11batch-with-overlap-sgd-nofreeze
#  --optimizer Adam \
#  --freeze 2 \

In [ ]:
#Display some sample images of the trained model.
#change the directory (custom14) depending on the latest train folder
from IPython.display import Image, display

display(Image(filename=f"{HOME}/yolov7/seg/runs/train-seg/120epochs-11batch-with-overlap-sgd-nofreeze/val_batch0_labels.jpg"))

#5. Start evalutating the model.

In [ ]:
#check directory
print(HOME)

In [ ]:
#Evaluation for images using the test folder in the dataset location which is the "Change the directory Compaction-Detection-1 if the milkfish fingerlings dataset is done".
from traitlets.traitlets import default
%cd {HOME}/yolov7/seg
!python segment/with-box-predict.py \
--weights {HOME}/yolov7/seg/runs/train-seg/120epochs-11batch-with-overlap-sgd-nofreeze/weights/best.pt \
--conf 0.25 \
--source {HOME}/yolov7/seg/evaluation-source/best-image-per-count \
--line-thickness 1 \
--name evaluation-with-box-0.25conf-best-image-per-count-120epochs-11batch-with-overlap-sgd-nofreeze \
--hide-labels \
--hide-conf
# --view-img

In [ ]:
#Displaying some of the images of evaluation result.
#Note: You need to change the filename in the directory below which is located before the ".jpg"
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/drive/MyDrive/v2-thesis-milkfish-fingerlings-counter/second-semester/milkfish-fingerlings-counter-development/v4MilkfishFingerlingsCounter/yolov7/seg/runs/predict-seg/change-this-using-the-filename/*.jpg')[:2]:
      display(Image(filename=imageName))
      print("\n")

#6. Starting to use the trained custom model.

In [ ]:
#1. Create a variable for the video path for the video source, name it VIDEO_PATH, example: VIDEO_PATH = f"{HOME}/yolov7/seg/early-blastocyst.mp4"
%cd {HOME}/yolov7/seg

VIDEO_PATH = f"{HOME}/yolov7/seg/video-test-source/9-confirmed-fingerlings.mp4"

In [ ]:
#2. Create a variable for the custom weights path, name it CUSTOM_WEIGHTS_PATH, example: CUSTOM_WEIGHTS_PATH = f"{HOME}/yolov7/seg/runs/train-seg/custom/weights/best.pt"
%cd {HOME}/yolov7/seg

CUSTOM_WEIGHTS_PATH = f"{HOME}/yolov7/seg/runs/train-seg/120epochs-11batch-with-overlap-sgd-nofreeze/weights/best.pt"

In [ ]:
#This will segment and process the video source and will create a separate video which is segmented. (Processed video is located at the given directory).
#3. %cd {HOME}/yolov7/seg !python segment/predict.py --weights  CUSTOMWEIGHTSPATH−−source VIDEO_PATH --name coco
%cd {HOME}/yolov7/seg
!python segment/with-box-predict.py --weights $CUSTOM_WEIGHTS_PATH \
--source $VIDEO_PATH \
--name video-demonstration-with-box-0.25conf-9confirmed-120epochs-11batch-with-overlap-sgd-nofreeze \
--line-thickness 1 \
--conf-thres 0.25 
# --hide-labels \
# --hide-conf \
# --save-txt \
# --view-img
# !python segment/no-bounding-box-predict.py --weights $CUSTOM_WEIGHTS_PATH \

#7. Accuracy Formula


In [ ]:
# Average Accuracy of 10 Different True Count

# Ten actual count numbers
true_count = [9, 20, 30, 42, 50, 60, 70, 80, 90, 100]
# Ten prediction count numbers
prediction_count = [9, 21, 28, 40, 48, 57, 67, 80, 82, 54]
# Calculate the accuracy percentage for each prediction
accuracies = []
for i in range(len(true_count)):
    if prediction_count[i] <= true_count[i]:
        	accuracy = prediction_count[i] / true_count[i] * 100
    else:
        	accuracy = (true_count[i] - abs(true_count[i] -prediction_count[i])) / true_count[i] * 100
    accuracies.append(accuracy)

# Calculate average accuracy percentage
avg_accuracy = sum(accuracies) / len(accuracies)

# Print results
print("Accuracy percentages:")
for i in range(len(accuracies)):
    print(f"Prediction {i+1}: {accuracies[i]:.2f}%")
print(f"Average accuracy: {avg_accuracy:.2f}%")

In [ ]:
# Average Accuracy of Single True Count with 10 Different Prediction

# Define the true count
true_count = 9

# Define the ten predicted counts
predicted_counts = [10, 8, 7, 9, 12, 10, 9, 8, 9, 11]

# Calculate the accuracy of each prediction as a percentage
accuracies = [100 * (1 - abs(predicted_count - true_count) / true_count) for predicted_count in predicted_counts]

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)

# Print the average accuracy as a percentage
print(f"The average accuracy is {average_accuracy:.2f}%")

In [ ]:
# Average Accuracy of Single True Count and Single Prediction

# Ten true count numbers
true_counts = [10]

# Ten prediction count numbers
#       true_counts = [9, 20, 30, 42, 50, 60, 70, 80, 90, 100]
prediction_counts = [11]

# Calculate accuracy percentage for each prediction
accuracies = []
for i in range(len(true_counts)):
    if prediction_counts[i] <= true_counts[i]:
        accuracy = prediction_counts[i] / true_counts[i] * 100
    else:
        accuracy = (2 * true_counts[i] - prediction_counts[i]) / true_counts[i] * 100
    accuracies.append(accuracy)

# Calculate average accuracy percentage
avg_accuracy = sum(accuracies) / len(accuracies)

# Print results
print("Accuracy percentages:")
for i in range(len(accuracies)):
    print(f"Prediction {i+1}: {accuracies[i]:.2f}%")
print(f"Average accuracy: {avg_accuracy:.2f}%")

End